In [1]:
import pickle
import json
import requests
import pandas as pd
from riotwatcher import LolWatcher, ApiError
import numpy as np
import matplotlib.pyplot as plt
# from time import sleep

with open('data/api-key.txt', 'r') as api:
    API_KEY = api.read()
lol_watcher = LolWatcher(API_KEY)

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4)"+
                    " AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135"+
                    " Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": API_KEY
}

CHAMP_ID = 64
TESTMATCH = "4748107995"

np.random.seed(7)

In [2]:
import os
import datetime

#import IPython
#import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

In [3]:
dfs_dict = pickle.load(open("data/timeline-di.p", "rb"))

In [4]:
gameids = list(dfs_dict.keys())
gameids.sort()
train_ids = gameids[0:int(len(gameids)*.8)]
val_ids = gameids[int(len(gameids)*.8):int(len(gameids)*.9)]
test_ids = gameids[int(len(gameids)*.9):]

In [5]:
dfs_dict[train_ids[1]][0:10]

,timestamp,0currentGold,1currentGold,2currentGold,3currentGold,4currentGold,5currentGold,6currentGold,7currentGold,8currentGold,...,2y,3y,4y,5y,6y,7y,8y,9y,player_x,player_y
0,0.0,500.0,500.0,500.0,500.0,500.0,500.0,500.0,500.0,500.0,...,293.0,471.0,649.0,14511.0,14291.0,14223.0,14401.0,14579.0,560.0,581.0
1,60032.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8318.0,2477.0,10387.0,11981.0,13198.0,9210.0,3006.0,2601.0,6219.0,8660.0
2,120042.0,112.0,121.0,112.0,63.0,23.0,63.0,121.0,84.0,63.0,...,7543.0,1949.0,1744.0,13040.0,6831.0,8043.0,2385.0,1938.0,2795.0,12410.0
3,180078.0,395.0,465.0,455.0,388.0,199.0,332.0,533.0,346.0,374.0,...,6613.0,1542.0,1452.0,13398.0,6545.0,6779.0,2093.0,3118.0,2426.0,13288.0
4,240098.0,732.0,151.0,49.0,697.0,394.0,662.0,1281.0,29.0,927.0,...,4568.0,3425.0,1858.0,13626.0,10304.0,2003.0,3107.0,14391.0,2808.0,13029.0
5,300112.0,74.0,561.0,360.0,326.0,241.0,1050.0,1480.0,668.0,48.0,...,6634.0,1822.0,6430.0,13354.0,12494.0,7082.0,8335.0,7092.0,3180.0,12949.0
6,360144.0,368.0,385.0,78.0,760.0,664.0,69.0,69.0,91.0,234.0,...,4902.0,2990.0,2551.0,14162.0,3069.0,6686.0,3628.0,3524.0,1424.0,11756.0
7,420153.0,738.0,592.0,675.0,966.0,269.0,431.0,389.0,262.0,1101.0,...,7320.0,2977.0,2835.0,11601.0,5632.0,8050.0,1223.0,5049.0,7888.0,7242.0
8,480178.0,973.0,881.0,101.0,185.0,183.0,961.0,297.0,367.0,544.0,...,1566.0,1396.0,1725.0,10175.0,9509.0,6825.0,3953.0,10348.0,1274.0,11185.0
9,540184.0,67.0,22.0,464.0,621.0,442.0,1517.0,625.0,801.0,834.0,...,7040.0,3266.0,3211.0,10685.0,8818.0,7610.0,4202.0,4630.0,996.0,8887.0


In [6]:
#train_ids[0]
test0 = dfs_dict[train_ids[0]][0:20].drop(["timestamp"],axis=1)
test1 = dfs_dict[train_ids[1]][0:20].drop(["timestamp"],axis=1)
column_indices = {name: i for i, name in enumerate(test0.columns)}

In [7]:
test0

,0currentGold,1currentGold,2currentGold,3currentGold,4currentGold,5currentGold,6currentGold,7currentGold,8currentGold,9currentGold,...,2y,3y,4y,5y,6y,7y,8y,9y,player_x,player_y
0,500.0,500.0,500.0,500.0,500.0,500.0,500.0,500.0,500.0,500.0,...,293.0,471.0,649.0,14511.0,14291.0,14223.0,14401.0,14579.0,560.0,361.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,...,2040.0,2934.0,1585.0,13791.0,7208.0,12340.0,6631.0,6206.0,1855.0,1338.0
2,84.0,121.0,63.0,21.0,23.0,84.0,121.0,91.0,21.0,48.0,...,6954.0,2664.0,3426.0,13392.0,11149.0,7424.0,5090.0,5090.0,2803.0,8078.0
3,188.0,753.0,346.0,80.0,243.0,596.0,478.0,448.0,107.0,102.0,...,7104.0,2438.0,1681.0,13198.0,10425.0,7732.0,3593.0,3386.0,4115.0,10079.0
4,103.0,1238.0,711.0,501.0,440.0,112.0,432.0,669.0,514.0,331.0,...,6376.0,4785.0,4803.0,13400.0,4863.0,7249.0,4933.0,4863.0,4858.0,8841.0
5,512.0,2162.0,137.0,139.0,168.0,416.0,390.0,73.0,123.0,30.0,...,8381.0,1700.0,6895.0,11652.0,9669.0,7860.0,2834.0,8403.0,8386.0,2265.0
6,903.0,40.0,184.0,557.0,539.0,651.0,646.0,346.0,468.0,193.0,...,5712.0,2517.0,2430.0,13919.0,6206.0,6538.0,3208.0,3208.0,7014.0,3194.0
7,1396.0,481.0,537.0,873.0,695.0,1037.0,273.0,608.0,835.0,393.0,...,5808.0,6561.0,7262.0,13925.0,6869.0,7360.0,5331.0,5331.0,11704.0,6544.0
8,1769.0,29.0,41.0,715.0,119.0,1257.0,196.0,267.0,82.0,63.0,...,7019.0,1468.0,6495.0,12145.0,10690.0,13813.0,4627.0,4627.0,2773.0,5242.0
9,62.0,669.0,384.0,1133.0,388.0,136.0,288.0,888.0,371.0,218.0,...,7013.0,2237.0,3061.0,12440.0,6494.0,7503.0,4476.0,4476.0,3016.0,8962.0


In [8]:
class WindowGenerator():
    def __init__(self, input_width, label_width, shift,
                 full_df=test0, val_df = test1,
                 dfdi=dfs_dict,
                 #train_df=train_df, val_df=val_df, test_df=test_df,
                 label_columns=None):
        # Store the raw data.
        #self.train_df = train_df
        #self.val_df = val_df
        #self.test_df = test_df
        self.full_df = full_df
        self.val_df = val_df
        self.dfdi = dfdi
        self.trainli = None
        self.valli = None
        self.testli = None

        # Work out the label column indices.
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in
                                                                        enumerate(label_columns)}
        self.column_indices = {name: i for i, name in
                                                     enumerate(full_df.columns)}

        # Work out the window parameters.
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def __repr__(self):
        return '\n'.join([
                f'Total window size: {self.total_window_size}',
                f'Input indices: {self.input_indices}',
                f'Label indices: {self.label_indices}',
                f'Label column name(s): {self.label_columns}'])

In [9]:
def split_window(self, features):
    inputs = features[:, self.input_slice, :]
    labels = features[:, self.labels_slice, :]
    if self.label_columns is not None:
        labels = tf.stack(
                [labels[:, :, self.column_indices[name]] for name in self.label_columns],
                axis=-1)
    

    # Slicing doesn't preserve static shape information, so set the shapes
    # manually. This way the `tf.data.Datasets` are easier to inspect.
    inputs.set_shape([None, self.input_width, None])
    labels.set_shape([None, self.label_width, None])

    return inputs, labels

WindowGenerator.split_window = split_window

In [10]:
def plot(self, model=None, plot_col="player_x", max_subplots=3):
    inputs, labels = self.example
    plt.figure(figsize=(12, 8))
    plot_col_index = self.column_indices[plot_col]
    print(plot_col_index, type(plot_col_index))
    max_n = min(max_subplots, len(inputs))
    for n in range(max_n):
        plt.subplot(3, 1, n+1)
        plt.ylabel(f'{plot_col} [normed]')
        plt.plot(self.input_indices, inputs[n, :, plot_col_index],
                         label='Inputs', marker='.', zorder=-10)

        if self.label_columns:
            print(type(self.label_columns_indices))
            label_col_index = self.label_columns_indices.get(plot_col, None)
            print(label_col_index)
        else:
            label_col_index = plot_col_index

        if label_col_index is None:
            continue

        print(self.label_indices)
        plt.scatter(self.label_indices, labels[n, :, label_col_index],
                                edgecolors='k', label='Labels', c='#2ca02c', s=64)
        if model is not None:
            predictions = model(inputs)
            plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                                    marker='X', edgecolors='k', label='Predictions',
                                    c='#ff7f0e', s=64)

        if n == 0:
            plt.legend()

    plt.xlabel('Time [h]')

WindowGenerator.plot = plot

In [11]:
def make_dataset(self, data, list_me=False):
    data = np.array(data, dtype=np.float32)
    ds = tf.keras.preprocessing.timeseries_dataset_from_array(
            data=data,
            targets=None,
            sequence_length=self.total_window_size,
            sequence_stride=1,
            shuffle=True,
            batch_size=32,)

    ds = ds.map(self.split_window)
    if list_me:
        return list(ds)
    return ds

WindowGenerator.make_dataset = make_dataset

In [12]:
def make_split_from_dfdi(self,splits=(.8,.9),minmax=(0,20)):
    # minmax is start and end time of window being sampled from.
    # splits is where the data is being split for train val test split
    if self.dfdi is None:
        print("no dfdi attached")
        raise KeyError("no dfdi attached")
    gameids = list(self.dfdi.keys())
    gameids.sort()
    train_ids = gameids[0:int(len(gameids)*splits[0])]
    val_ids = gameids[int(len(gameids)*splits[0]):int(len(gameids)*splits[1])]
    test_ids = gameids[int(len(gameids)*splits[1]):]
    if self.trainli is None:
        trainli = build_batch(self, self.dfdi, train_ids, minmax)
        train_feats = tf.concat([batch[0] for batch in trainli], axis=0)
        train_labels = tf.concat([batch[1] for batch in trainli], axis=0)
        
    if self.valli is None:
        valli = build_batch(self, self.dfdi, val_ids, minmax)
        val_feats = tf.concat([batch[0] for batch in valli], axis=0)
        val_labels = tf.concat([batch[1] for batch in valli], axis=0)
            
    if self.testli is None:
        testli = build_batch(self, self.dfdi, test_ids, minmax)
        test_feats = tf.concat([batch[0] for batch in testli], axis=0)
        test_labels = tf.concat([batch[1] for batch in testli], axis=0)
            
        
    print(train_feats.shape, train_labels.shape)
    self.train = (train_feats, train_labels)
    self.val = (val_feats, val_labels)
    self.test = (test_feats, test_labels)

WindowGenerator.make_split_from_dfdi = make_split_from_dfdi

In [13]:
@property
def example(self):
    """Get and cache an example batch of `inputs, labels` for plotting."""
    result = getattr(self, '_example', None)
    if result is None:
        # No example batch was found, so get one from the `.train` dataset
        result = next(iter(self.train))
        # And cache it for next time
        self._example = result
    return result

WindowGenerator.example = example

In [14]:
def build_batch(self, dfdi, id_li, minmax):
    dataset_li = []
    for match_id in id_li:
        df = dfdi[match_id]
        df = df.drop(["timestamp"], axis=1)
        df = df.iloc[minmax[0]:minmax[1]]
        dataset_li = dataset_li + make_dataset(self, data=df, list_me=True)
    return dataset_li

WindowGenerator.build_batch = build_batch

In [15]:
test0.columns

Index(['0currentGold', '1currentGold', '2currentGold', '3currentGold',
       '4currentGold', '5currentGold', '6currentGold', '7currentGold',
       '8currentGold', '9currentGold', '0totalGold', '1totalGold',
       '2totalGold', '3totalGold', '4totalGold', '5totalGold', '6totalGold',
       '7totalGold', '8totalGold', '9totalGold', '0level', '1level', '2level',
       '3level', '4level', '5level', '6level', '7level', '8level', '9level',
       '0jungleMinionsKilled', '1jungleMinionsKilled', '2jungleMinionsKilled',
       '3jungleMinionsKilled', '4jungleMinionsKilled', '5jungleMinionsKilled',
       '6jungleMinionsKilled', '7jungleMinionsKilled', '8jungleMinionsKilled',
       '9jungleMinionsKilled', '0x', '2x', '3x', '4x', '5x', '6x', '7x', '8x',
       '9x', '0y', '2y', '3y', '4y', '5y', '6y', '7y', '8y', '9y', 'player_x',
       'player_y'],
      dtype='object')

In [16]:
w2 = WindowGenerator(input_width=5, label_width=2, shift=2,
                     label_columns=["player_x","player_y"])
w2

Total window size: 7
Input indices: [0 1 2 3 4]
Label indices: [5 6]
Label column name(s): ['player_x', 'player_y']

In [17]:
class Baseline(tf.keras.Model):
    def __init__(self, label_index=None):
        super().__init__()
        self.label_index = label_index

    def call(self, inputs):
        if self.label_index is None:
            return inputs
        result = inputs[:, :, self.label_index]
        return result[:, :, tf.newaxis]

In [18]:
single_step_window = WindowGenerator(
    input_width=1, label_width=1, shift=1,
    label_columns=["player_x"])#,"player_y"
single_step_window

Total window size: 2
Input indices: [0]
Label indices: [1]
Label column name(s): ['player_x']

In [19]:
single_step_window.make_split_from_dfdi()

(1824, 1, 60) (1824, 1, 1)


In [20]:
train_hold = single_step_window.train# = tf.data.Dataset.from_tensor_slices([train_feats, train_labels])
val_hold = single_step_window.train

In [21]:
a = tf.data.Dataset.from_tensor_slices(train_hold)

In [22]:
baseline = Baseline()
baseline.compile(loss=tf.losses.MeanSquaredError(),
                 metrics=[tf.metrics.MeanAbsoluteError()])

In [23]:
baseline.fit(x=train_hold[0],
           y=train_hold[1],
           batch_size=32,
           epochs=30,
           verbose=1,
           validation_data=val_hold
          )

Epoch 1/30
57/57 [==============================] - 0s 5ms/step - loss: 55685020.0000 - mean_absolute_error: 6384.0645 - val_loss: 55685016.0000 - val_mean_absolute_error: 6384.0654
Epoch 2/30
57/57 [==============================] - 0s 3ms/step - loss: 55685020.0000 - mean_absolute_error: 6384.0645 - val_loss: 55685016.0000 - val_mean_absolute_error: 6384.0654
Epoch 3/30
57/57 [==============================] - 0s 2ms/step - loss: 55685016.0000 - mean_absolute_error: 6384.0659 - val_loss: 55685016.0000 - val_mean_absolute_error: 6384.0654
Epoch 4/30
57/57 [==============================] - 0s 2ms/step - loss: 55685008.0000 - mean_absolute_error: 6384.0635 - val_loss: 55685016.0000 - val_mean_absolute_error: 6384.0654
Epoch 5/30
57/57 [==============================] - 0s 3ms/step - loss: 55685016.0000 - mean_absolute_error: 6384.0654 - val_loss: 55685016.0000 - val_mean_absolute_error: 6384.0654
Epoch 6/30
57/57 [==============================] - 0s 3ms/step - loss: 55685020.0000 - me

In [24]:
linear = tf.keras.Sequential([
    tf.keras.layers.Dense(units=1)
])

In [25]:
linear.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])


In [26]:
linear.fit(x=train_hold[0],
           y=train_hold[1],
           batch_size=32,
           epochs=30,
           verbose=1,
           validation_data=val_hold
          )

Epoch 1/30
57/57 [==============================] - 0s 4ms/step - loss: 24856382.0000 - mean_absolute_error: 4030.5798 - val_loss: 20724558.0000 - val_mean_absolute_error: 3681.1848
Epoch 2/30
57/57 [==============================] - 0s 6ms/step - loss: 19394070.0000 - mean_absolute_error: 3586.0774 - val_loss: 18000588.0000 - val_mean_absolute_error: 3467.2251
Epoch 3/30
57/57 [==============================] - 0s 7ms/step - loss: 16999886.0000 - mean_absolute_error: 3377.3975 - val_loss: 16022699.0000 - val_mean_absolute_error: 3291.3506
Epoch 4/30
57/57 [==============================] - 0s 2ms/step - loss: 15306362.0000 - mean_absolute_error: 3222.2551 - val_loss: 14666114.0000 - val_mean_absolute_error: 3146.6174
Epoch 5/30
57/57 [==============================] - 0s 2ms/step - loss: 14163464.0000 - mean_absolute_error: 3091.4434 - val_loss: 13620480.0000 - val_mean_absolute_error: 3029.8252
Epoch 6/30
57/57 [==============================] - 0s 1ms/step - loss: 13257179.0000 - me

In [27]:
test_hold = single_step_window.test

In [28]:
linear_test_pred = linear.predict(x=test_hold[0])
baseline_test_pred = baseline.predict(x=test_hold[0])

In [29]:
linear_error = linear.evaluate(test_hold[0], test_hold[1])#, batch_size=128)
baseline_error = baseline.evaluate(test_hold[0], test_hold[1])
print("baseline mae error:", baseline_error[1])
print("linear mae error:", linear_error[1])

8/8 [==============================] - 0s 6ms/step - loss: 60570472.0000 - mean_absolute_error: 6523.5928
baseline mae error: 6523.5927734375
linear mae error: 2793.60888671875


In [30]:
wide_window = WindowGenerator(
    input_width=10, label_width=10, shift=1,
    label_columns=["player_x"])

wide_window

Total window size: 11
Input indices: [0 1 2 3 4 5 6 7 8 9]
Label indices: [ 1  2  3  4  5  6  7  8  9 10]
Label column name(s): ['player_x']